In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

pd.set_option('max_column', 100)

In [3]:
#Function to replace spaces with underscores and removing \xa0
def clean_col(c):
    return c.lower().strip().replace(" ", "_").replace("\xa0", "")

In [4]:
raw_path = '../../dataset/raw/'
mst_path = '../../dataset/mst/'

In [5]:
os.listdir(raw_path)

['Participant_Survey.xlsx']

In [6]:
df = pd.read_excel(raw_path + "Participant_Survey.xlsx")
df.head(2)

,Respondent ID,Collector ID,Start Date,End Date,IP Address,Email Address,First Name,Last Name,Custom Data 1,"What brand/type of E-bike did you buy (e.g., Sonders X)?",How much did your E-bike cost (before the rebate)?,"Overall, how satisfied are you with your E-bike?","Why do you like or dislike your E-bike? What, if any, other thoughts would you like to share about your E-bike?",How much did the $700 rebate influence your decision to purchase an E-bike?,How far is your average trip by E-bike?,For what purpose are MOST of your E-bike rides?,Why did you purchase an E-bike? Select all that apply.,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,How did you hear about the E-bike rebate program? Select all that apply.,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,How familiar were you with E-bikes before hearing about this rebate program?,Which of the following do you have at home? (Select all that apply),Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Select your top 3 most-used modes of transportation BEFORE purchasing an E-Bike.,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Select your top 3 most-used modes of transportation AFTER purchasing an E-Bike.,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,How often did you bicycle for any purpose BEFORE purchasing an E-bike?,How often do you bicycle for any purpose SINCE purchasing an E-bike?,Do you use your E-bike for some trips that you used to take by car?,"Have you used your E-bike on a river path or ‘greenway’ in the past 12 months? (e.g., San Gabriel River or Rio Hondo River trails)",How easy or difficult is it to bicycle in your community?,Did your level of comfort riding a bicycle in traffic increase after taking the City Cycling Class?,How safe do you feel while using the following types of bike facilities? (See images below for reference),Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,How safe do you feel riding a bicycle in traffic?,"What barriers to using your E-bike have you encountered thus far, if any?",How many members of your household use your E-bike at least once a month?,"Have you allowed any friends, family, colleagues, neighbors, or others to try using your E-bike? If so, how many individuals other than you?",Unnamed: 69,Would you like to see future E-bike rebate programs?,Should future programs expand the list of eligible light electric vehicles to include electric-scooters and similar micro-mobility devices?,"Other suggestions, feedback, or comments for project organizers.",Community of Residence,Zip Code,Number of persons in household,Household income,What is your age group?,"Do you own any other Electric Vehicles (EVs)? If so, please select all that apply.",Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Ethnicity/Race (select all that apply),Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,What gender do you identify with?,Unnamed: 94,Unnamed: 95,Unnamed: 96
0,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,Open-Ended Response,Response,Response,Open-Ended Response,Response,Response,Response,Business/Work Use,Injury or Disability,Sustainability / Environmental Benefits,Cargo Hauling,Recreation or Exercise,Transport children,Save money / reduce transportation costs,Hills,Other (please specify),"Social Media (e.g., Facebook, Instagram, Twitter)",News article,BikeSGV listserve/email,Friend/family/colleague,Flyer,Poster,Other (please specify),Response,Bicycle (regular),Electric Bicycle (E-bike),Working motor vehicle,Scooter,Motorcycle,None of the above,Walked,Biked,Skated / Scooted,Used public transit,Carpooled / rode as a passenger,Used ride share service (Lyft/Uber),Drove car / motorcycle,Walked,Biked,Skated / Scooted,Used public transit,Carpooled / rode as a passenger,Used ride share service (Lyft/Uber),Drove car / motorcycle,Response,Response,Response,Response,Respon

In [18]:
# Get the column names, the headers that contains "unnamed", and values of the second row and their corresponding indice
col_names = list(df.columns)
unnameds = [x for x in col_names if 'Unnamed' in x]
sec_row = df.loc[0, ].values
inds = [col_names.index(x) for x in unnameds]

print(col_names)

# Add the indices of columns that has Unnamed in there
temp = []
for i in range(len(inds) - 1):
    value = inds[i]
    next_value = inds[i+1]
    temp.append(value)
    if value < next_value - 1:
        temp.append(next_value-1)  
temp.append(next_value)

# Add in the first and last indice by hand
temp.append(16)

# Replace column names by second row values whenever needed     
for i in range(len(col_names)):
    if i in temp:
        col_names[i] = sec_row[i]

['respondent_id', 'collector_id', 'start_date', 'end_date', 'ip_address', 'email_address', 'first_name', 'last_name', 'custom_data_1', 'what_brand/type_of_e-bike_did_you_buy_(e.g.,_sonders_x)?', 'how_much_did_your_e-bike_cost_(before_the_rebate)?', 'overall,_how_satisfied_are_you_with_your_e-bike?', 'why_do_you_like_or_dislike_your_e-bike?_what,_if_any,_other_thoughts_would_you_like_to_share_about_your_e-bike?', 'how_much_did_the_$700_rebate_influence_your_decision_to_purchase_an_e-bike?', 'how_far_is_your_average_trip_by_e-bike?', 'for_what_purpose_aremost_of_your_e-bike_rides?', 'business/work_use', 'injury_or_disability', 'sustainability_/_environmental_benefits', 'cargo_hauling', 'recreation_or_exercise', 'transport_children', 'save_money_/_reducetransportation_costs', 'hills', 'other_(please_specify)', 'social_media_(e.g.,_facebook,_instagram,_twitter)', 'news_article', 'bikesgv_listserve/email', 'friend/family/colleague', 'flyer', 'poster', 'other_(please_specify)', 'how_familiar

In [8]:
# Create a question look-up table
q_map = defaultdict(list)

# retrieve questions that span multiple columns
q_list = [c for c in df.columns if c not in col_names]

question = q_list[0]
for i in range(len(q_list)):
    if 'Unname' in q_list[i]:
        q_list[i] = question
    else:
        question = q_list[i]

# make a list of the responses to the multi-column questions
for i in temp:
    q_map['ans_choice'].append(sec_row[i])

# put the two parts, q_list and q_map together
q_map['question'] = q_list
q_map = pd.DataFrame(q_map)
q_map.head()

,ans_choice,question
0,Injury or Disability,Why did you purchase an E-bike? Select all tha...
1,Sustainability / Environmental Benefits,Why did you purchase an E-bike? Select all tha...
2,Cargo Hauling,Why did you purchase an E-bike? Select all tha...
3,Recreation or Exercise,Why did you purchase an E-bike? Select all tha...
4,Transport children,Why did you purchase an E-bike? Select all tha...


In [9]:
# saving q_map to a csv file
q_map.to_csv(mst_path + 'question_lookup.csv', index = None)
q_map = pd.read_csv(mst_path + 'question_lookup.csv')
q_map.head()

,ans_choice,question
0,Injury or Disability,Why did you purchase an E-bike? Select all tha...
1,Sustainability / Environmental Benefits,Why did you purchase an E-bike? Select all tha...
2,Cargo Hauling,Why did you purchase an E-bike? Select all tha...
3,Recreation or Exercise,Why did you purchase an E-bike? Select all tha...
4,Transport children,Why did you purchase an E-bike? Select all tha...


In [13]:
# Replace columns names 
col_names = [clean_col(c) for c in col_names]
df.columns = col_names

# Delete second row
df = df.loc[-1, ]

KeyError: 'the label [-1] is not in the [index]'

In [14]:
df.head()

,respondent_id,collector_id,start_date,end_date,ip_address,email_address,first_name,last_name,custom_data_1,"what_brand/type_of_e-bike_did_you_buy_(e.g.,_sonders_x)?",how_much_did_your_e-bike_cost_(before_the_rebate)?,"overall,_how_satisfied_are_you_with_your_e-bike?","why_do_you_like_or_dislike_your_e-bike?_what,_if_any,_other_thoughts_would_you_like_to_share_about_your_e-bike?",how_much_did_the_$700_rebate_influence_your_decision_to_purchase_an_e-bike?,how_far_is_your_average_trip_by_e-bike?,for_what_purpose_aremost_of_your_e-bike_rides?,business/work_use,injury_or_disability,sustainability_/_environmental_benefits,cargo_hauling,recreation_or_exercise,transport_children,save_money_/_reducetransportation_costs,hills,other_(please_specify),"social_media_(e.g.,_facebook,_instagram,_twitter)",news_article,bikesgv_listserve/email,friend/family/colleague,flyer,poster,other_(please_specify),how_familiar_were_you_with_e-bikes_before_hearing_about_this_rebate_program?,bicycle_(regular),electric_bicycle_(e-bike),working_motor_vehicle,scooter,motorcycle,none_of_the_above,walked,biked,skated_/_scooted,used_public_transit,carpooled_/_rode_as_a_passenger,used_ride_share_service_(lyft/uber),drove_car_/_motorcycle,walked,biked,skated_/_scooted,used_public_transit,carpooled_/_rode_as_a_passenger,used_ride_share_service_(lyft/uber),drove_car_/_motorcycle,how_oftendid_you_bicycle_for_any_purposebefore_purchasing_an_e-bike?,how_often_do_you_bicyclefor_any_purposesince_purchasing_an_e-bike?,do_you_use_your_e-bike_for_some_trips_that_you_used_to_take_by_car?,"have_youused_your_e-bikeon_a_river_path_or_‘greenway’_in_the_past_12_months?_(e.g.,_san_gabriel_river_or_rio_hondo_river_trails)",how_easy_or_difficult_is_it_to_bicycle_in_your_community?,did_your_level_of_comfort_riding_a_bicycle_in_traffic_increase_after_taking_the_city_cycling_class?,off-street_greenways_/_multi-use_paths,on-street_separated_/_physically_‘protected’_bikeways,buffered_bike_lanes,color_painted_bike_lanes,striped_bike_lanes,"bike_routes_(signage,_sharrows)",how_safe_doyou_feel_riding_a_bicycle_in_traffic?,"what_barriers_to_using_your_e-bike_have_you_encountered_thus_far,_if_any?",how_many_members_of_your_household_use_your_e-bike_at_least_once_a_month?,response,other_(please_specify),would_you_like_to_see_future_e-bike_rebate_programs?,should_future_programsexpand_the_list_of_eligible_light_electric_vehicles_to_include_electric-scooters_and_similar_micro-mobility_devices?,"other_suggestions,_feedback,_or_comments_forproject_organizers.",community_of_residence,zip_code,number_of_persons_in_household,household_income,what_is_your_age_group?,electric_car,electric_motorcycle,electric_scooter,electric_skateboard,electric_wheel,"no,_i_do_not_own_other_evs",other_(please_specify),american_indian_or_alaska_native,asian/pacific_islander,black_or_african_american,hispanic_or_latino,white,multi-ethnic,decline_to_state,other_(please_specify),female,male,decline_to_state,other
0,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,Open-Ended Response,Response,Response,Open-Ended Response,Response,Response,Response,Business/Work Use,Injury or Disability,Sustainability / Environmental Benefits,Cargo Hauling,Recreation or Exercise,Transport children,Save money / reduce transportation costs,Hills,Other (please specify),"Social Media (e.g., Facebook, Instagram, Twitter)",News article,BikeSGV listserve/email,Friend/family/colleague,Flyer,Poster,Other (please specify),Response,Bicycle (regular),Electric Bicycle (E-bike),Working motor vehicle,Scooter,Motorcycle,None of the above,Walked,Biked,Skated / Scooted,Used public transit,Carpooled / rode as a passenger,Used ride share service (Lyft/Uber),Drove car / motorcycle,Walked,Biked,Skated / Scooted,Used public transit,Carpooled / rode as a passenger,Used ride share service (Lyft/Uber),Drove car / motorcycle,Response,Response,Response,Response,Response,Response,Off-street greenways / multi-use paths,On-street separated / physically ‘protected’ b...,Buffered bike la

In [15]:
df.to_csv(mst_path + 'Participant_Survey.csv', index = None)

In [14]:
# proportion of responses are NaN
df.isna().sum()/len(df)

Respondent ID                                                                                                                                   0.032258
Collector ID                                                                                                                                    0.032258
Start Date                                                                                                                                      0.032258
End Date                                                                                                                                        0.032258
IP Address                                                                                                                                      0.032258
Email Address                                                                                                                                   1.000000
First Name                                                                        

In [83]:
df = df.drop(['first_name'], 1)
df.columns

Index(['respondent_id', 'collector_id', 'start_date', 'end_date', 'ip_address',
       'email_address', 'last_name', 'custom_data_1',
       'what_brand/type_of_e-bike_did_you_buy_(e.g.,_sonders_x)?',
       'how_much_did_your_e-bike_cost_(before_the_rebate)?',
       'overall,_how_satisfied_are_you_with_your_e-bike?',
       'why_do_you_like_or_dislike_your_e-bike?_what,_if_any,_other_thoughts_would_you_like_to_share_about_your_e-bike?',
       'how_much_did_the_$700_rebate_influence_your_decision_to_purchase_an_e-bike?',
       'how_far_is_your_average_trip_by_e-bike?',
       'for_what_purpose_aremost_of_your_e-bike_rides?', 'business/work_use',
       'injury_or_disability', 'sustainability_/_environmental_benefits',
       'cargo_hauling', 'recreation_or_exercise', 'transport_children',
       'save_money_/_reducetransportation_costs', 'hills',
       'other_(please_specify)',
       'social_media_(e.g.,_facebook,_instagram,_twitter)', 'news_article',
       'bikesgv_listserve/ema